<a href="https://colab.research.google.com/github/sankalp2004/Hate-Speech-Detection-/blob/main/HateSpeechShown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install datasets

from datasets import load_dataset

# Load the dataset
dataset = load_dataset("tdavidson/hate_speech_offensive")

# Convert the Hugging Face Dataset to a pandas DataFrame
data = dataset['train'].to_pandas()

# Check unique classes in the original dataset
class_counts = data['class'].value_counts()
print("Class Counts in the Dataset:")
print(class_counts)

# Display the first few rows of the DataFrame
print(data.head())


  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached datasets-3.1.0-py3-none-any.whl (480 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following de

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24783 [00:00<?, ? examples/s]

Class Counts in the Dataset:
class
1    19190
2     4163
0     1430
Name: count, dtype: int64
   count  hate_speech_count  offensive_language_count  neither_count  class  \
0      3                  0                         0              3      2   
1      3                  0                         3              0      1   
2      3                  0                         3              0      1   
3      3                  0                         2              1      1   
4      6                  0                         6              0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
from sklearn.model_selection import train_test_split

df = data
# Split the dataset into 80% training and 20% testing
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Make copies if needed
data = train_data.copy()
test_data = test_data.copy()

print(f"Training data size: {len(data)}")
print(f"Testing data size: {len(test_data)}")


Training data size: 19826
Testing data size: 4957


# New Section

In [ ]:
data.isnull().values.any()


False

In [ ]:
import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn import svm
from sklearn.metrics import precision_recall_fscore_support

# Function to process text
def text_processing(raw_text):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_text)  # Remove non-letters
    words = letters_only.lower().split()  # Convert to lowercase and split
    stops = set(stopwords.words("english"))  # Define stopwords
    meaningful_words = [w for w in words if w not in stops]  # Remove stopwords
    return " ".join(meaningful_words)  # Join words back into a single string

# Processing text in the training data
clean_text = []
for text in data["tweet"]:  # Use 'tweet' instead of 'text'
    clean_text.append(text_processing(text))
data["clean_text"] = clean_text  # Add processed text as a new column

# Processing text in the testing data
clean_text_test = []
for text in test_data["tweet"]:  # Use 'tweet' for the test data as well
    clean_text_test.append(text_processing(text))
test_data["clean_text"] = clean_text_test  # Add processed text as a new column


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'class' is the column that contains the labels
X_train, X_test, Y_train, Y_test = train_test_split(
    data["clean_text"],  # Input features
    data["class"],       # Output labels
    test_size=0.2,      # 20% for testing
    random_state=42,    # For reproducibility
    stratify=data["class"]  # Ensures proportional representation of classes in train/test sets
)

# Optional: Print shapes to confirm split
print("Training set size:", len(X_train), "Testing set size:", len(X_test))
print("Classes in training set:", Y_train.value_counts())
print("Classes in testing set:", Y_test.value_counts())


Training set size: 15860 Testing set size: 3966
Classes in training set: class
1    12286
2     2662
0      912
Name: count, dtype: int64
Classes in testing set: class
1    3072
2     666
0     228
Name: count, dtype: int64


In [ ]:
# Initialize CountVectorizer
vectorizer = CountVectorizer(
    analyzer="word",        # Analyzing individual words
    tokenizer=None,         # Using the default tokenization
    preprocessor=None,      # No preprocessor, since preprocessing is done already
    stop_words=None,        # Using no stop words; they were removed during preprocessing
    max_features=5000       # Limiting to 5000 features
)

# Fit and transform the training data
train_data_features = vectorizer.fit_transform(X_train).toarray()

# Transform the testing data
test_data_features = vectorizer.transform(X_test).toarray()


In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# SVM with linear kernel
clf = svm.SVC(kernel='linear', C=1.0)

# Training
print("Training")
clf.fit(train_data_features, Y_train)

# Testing
print("Testing")
predicted = clf.predict(test_data_features)

# Calculate accuracy
accuracy = np.mean(predicted == Y_test)
print("Accuracy: ", accuracy)

# Calculate additional metrics
precision, recall, f1, _ = precision_recall_fscore_support(Y_test, predicted, average='weighted')
print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


Training
Testing
Accuracy:  0.899899142713061
Precision: 0.89392337796273, Recall: 0.899899142713061, F1 Score: 0.8965014064946625


In [ ]:
import numpy as np

# Update the 'class' column for the training data
data['class'] = np.where(data['class'] == 1, 'hate_speech',
                          np.where(data['class'] == 2, 'offensive_language', 'neither'))

# Update the 'class' column for the test data
test_data['class'] = np.where(test_data['class'] == 1, 'hate_speech',
                               np.where(test_data['class'] == 2, 'offensive_language', 'neither'))

# Extract target values for training and testing sets
y_train = data['class'].values
y_test = test_data['class'].values

# Optional: Print to confirm the updates
print("Unique labels in training data:", np.unique(y_train))
print("Unique labels in test data:", np.unique(y_test))


Unique labels in training data: ['hate_speech' 'neither' 'offensive_language']
Unique labels in test data: ['hate_speech' 'neither' 'offensive_language']


In [ ]:
# Install NLTK and download the necessary resources
!pip install nltk
import nltk

# Download the 'punkt' resource
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# Define the text column to tokenize
text_column = 'tweet'  # Actual column name containing text data

# Tokenize the specified text column in the main data and test data
data['tokenized_sents'] = data[text_column].apply(lambda x: word_tokenize(str(x)))
test_data['tokenized_sents'] = test_data[text_column].apply(lambda x: word_tokenize(str(x)))

# Check if df exists and tokenize if it does
if 'df' in locals():  # Check if df exists
    df['tokenized_sents'] = df[text_column].apply(lambda x: word_tokenize(str(x)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from gensim.models import Word2Vec

# Define training data
sentences = df['tokenized_sents']

# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=200, window=4, min_count=1, sg=1)

# Instead of model.wv.vocab, use model.wv.key_to_index to get the vocabulary
words = list(model.wv.key_to_index.keys())  # Get keys from key_to_index dictionary

# Save the trained model
model.save('model.bin')

In [ ]:
import numpy as np
from gensim.models import Word2Vec

# Define vocab_size based on the length of the words list
vocab_size = len(words)

# Initialize the embedding matrix with zeros
embedding_matrix = np.zeros((vocab_size, 200))

for i in range(vocab_size):  # Iterate through the vocabulary
    # Use model.wv[words[i]] to get the embedding vector
    embedding_vector = model.wv[words[i]]
    embedding_matrix[i] = embedding_vector  # Assign the embedding vector to the matrix

# Optionally, print the embedding matrix shape
print("Embedding matrix shape:", embedding_matrix.shape)


Embedding matrix shape: (42410, 200)


In [ ]:
!pip install -q keras_preprocessing
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.1 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import load_model

In [ ]:
# Initialize the tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the training data's tokenized sentences
tokenizer.fit_on_texts(data['tokenized_sents'])
sequences = tokenizer.texts_to_sequences(data['tokenized_sents'])
X_t = pad_sequences(sequences, maxlen=50)

# Define vocab_size based on the tokenizer's word index
vocab_size = len(tokenizer.word_index) + 1  # +1 for zero padding

# Prepare test data sequences
test_sequences = tokenizer.texts_to_sequences(test_data['tokenized_sents'])
X_test = pad_sequences(test_sequences, maxlen=50)

# Optionally, print the shapes of your padded sequences
print("Training data shape:", X_t.shape)
print("Testing data shape:", X_test.shape)


Training data shape: (19826, 50)
Testing data shape: (4957, 50)


In [ ]:
# Count occurrences of each class in the data DataFrame
class_counts = data['class'].value_counts()

# Count occurrences of each class in the test_data DataFrame
class_counts_test = test_data['class'].value_counts()

# Print the counts
print("Class Counts in Data:")
print(class_counts)

print("\nClass Counts in Test Data:")
print(class_counts_test)

# Get the total counts of classes
count1 = class_counts.tolist()  # Convert the counts to a list
count_test = class_counts_test.tolist()  # Convert the counts to a list

# Initialize ratios for data
total_data = len(data)
ratio_hate_speech = (data['class'] == 1).sum() / total_data if total_data > 0 else 0
ratio_offensive_language = (data['class'] == 2).sum() / total_data if total_data > 0 else 0
ratio_neither = (data['class'] == 0).sum() / total_data if total_data > 0 else 0

# Create columns for ratios in the original DataFrame
data['ratio_hate_speech'] = ratio_hate_speech
data['ratio_offensive_language'] = ratio_offensive_language
data['ratio_neither'] = ratio_neither

# For test_data
total_test_data = len(test_data)
ratio_hate_speech_test = (test_data['class'] == 1).sum() / total_test_data if total_test_data > 0 else 0
ratio_offensive_language_test = (test_data['class'] == 2).sum() / total_test_data if total_test_data > 0 else 0
ratio_neither_test = (test_data['class'] == 0).sum() / total_test_data if total_test_data > 0 else 0

# Create columns for ratios in the test DataFrame
test_data['ratio_hate_speech'] = ratio_hate_speech_test
test_data['ratio_offensive_language'] = ratio_offensive_language_test
test_data['ratio_neither'] = ratio_neither_test

# Optional: Print the updated DataFrames to check the new columns
print(data[['class', 'ratio_hate_speech', 'ratio_offensive_language', 'ratio_neither']])
print(test_data[['class', 'ratio_hate_speech', 'ratio_offensive_language', 'ratio_neither']])


Class Counts in Data:
class
hate_speech           15358
offensive_language     3328
neither                1140
Name: count, dtype: int64

Class Counts in Test Data:
class
hate_speech           3832
offensive_language     835
neither                290
Name: count, dtype: int64
                    class  ratio_hate_speech  ratio_offensive_language  \
15272             neither                0.0                       0.0   
9351   offensive_language                0.0                       0.0   
20323         hate_speech                0.0                       0.0   
3638          hate_speech                0.0                       0.0   
20579         hate_speech                0.0                       0.0   
...                   ...                ...                       ...   
21575  offensive_language                0.0                       0.0   
5390          hate_speech                0.0                       0.0   
860           hate_speech                0.0           

In [ ]:
import pandas as pd
import numpy as np

# Assuming 'data' and 'test_data' are your DataFrames

# Count occurrences of each label in the data DataFrame
label_counts = data['class'].value_counts()
label_counts_test = test_data['class'].value_counts()

# Print the counts
print("Label Counts in Data:")
print(label_counts)

print("\nLabel Counts in Test Data:")
print(label_counts_test)

# Get the total counts of labels
count1 = label_counts.tolist() if not label_counts.empty else [0, 0, 0]  # Handle empty case
count_test = label_counts_test.tolist() if not label_counts_test.empty else [0, 0, 0]  # Handle empty case

# Initialize ratios for data
total_data = len(data)
ratio_positive = np.zeros(total_data)
ratio_negative = np.zeros(total_data)
ratio_offensive = np.zeros(total_data)

# Calculate ratios for the data DataFrame
if total_data > 0:  # Only calculate if data is not empty
    ratio_positive = (data['class'] == 1).mean()
    ratio_negative = (data['class'] == 0).mean()
    ratio_offensive = (data['class'] == 2).mean()

    # Create columns for ratios in the original DataFrame
    data['ratio_positive'] = ratio_positive
    data['ratio_negative'] = ratio_negative
    data['ratio_offensive'] = ratio_offensive

# For test_data
total_test_data = len(test_data)
ratio_positive_test = np.zeros(total_test_data)
ratio_negative_test = np.zeros(total_test_data)
ratio_offensive_test = np.zeros(total_test_data)

# Calculate ratios for the test_data DataFrame
if total_test_data > 0:  # Only calculate if test_data is not empty
    ratio_positive_test = (test_data['class'] == 1).mean()
    ratio_negative_test = (test_data['class'] == 0).mean()
    ratio_offensive_test = (test_data['class'] == 2).mean()

    # Create columns for ratios in the test DataFrame
    test_data['ratio_positive'] = ratio_positive_test
    test_data['ratio_negative'] = ratio_negative_test
    test_data['ratio_offensive'] = ratio_offensive_test

# Optional: Print the updated DataFrames to check the new columns
print("\nData DataFrame with Ratios:")
print(data[['class', 'ratio_positive', 'ratio_negative', 'ratio_offensive']])

print("\nTest DataFrame with Ratios:")
print(test_data[['class', 'ratio_positive', 'ratio_negative', 'ratio_offensive']])

# Initialize tendency columns based on labels for the training data
if total_data > 0:  # Ensure there is data
    data['tendency_positive'] = (data['class'] == 1).mean()  # Overall tendency for 1s (positive)
    data['tendency_negative'] = (data['class'] == 0).mean()  # Overall tendency for 0s (negative)
    data['tendency_offensive'] = (data['class'] == 2).mean()  # Overall tendency for 2s (offensive)

# For the test_data
if total_test_data > 0:  # Ensure there is test data
    test_data['tendency_positive'] = (test_data['class'] == 1).mean()  # Overall tendency for 1s (positive)
    test_data['tendency_negative'] = (test_data['class'] == 0).mean()  # Overall tendency for 0s (negative)
    test_data['tendency_offensive'] = (test_data['class'] == 2).mean()  # Overall tendency for 2s (offensive)

# Optional: Print the updated DataFrames to check the new tendency columns
print("\nData DataFrame with Tendencies:")
print(data[['class', 'tendency_positive', 'tendency_negative', 'tendency_offensive']])

print("\nTest DataFrame with Tendencies:")
print(test_data[['class', 'tendency_positive', 'tendency_negative', 'tendency_offensive']])

# Now let's assume you have your features ready, and you want to concatenate the datasets
# Example feature arrays (replace with your actual feature arrays)
# X_t should have shape (total_data, number_of_features)
# tendency_hate and tendency_no_hate should have shape (total_data, 1)

X_t = np.random.rand(total_data, 51)  # Replace with your actual features
tendency_hate = data[['tendency_positive']].values  # Shape (total_data, 1)
tendency_no_hate = data[['tendency_negative']].values  # Shape (total_data, 1)
tendency_offensive = data[['tendency_offensive']].values  # Shape (total_data, 1)

# Concatenate features for the training data
X1_t = np.concatenate((X_t, tendency_hate, tendency_no_hate, tendency_offensive), axis=1)
print("Shape of Training Data with Tendencies:", X1_t.shape)  # Should be (total_data, 54)

# For the test data
# Assuming you have similar features for test_data
X_test = np.random.rand(total_test_data, 51)  # Replace with your actual test features
tendency_hate_test = test_data[['tendency_positive']].values  # Shape (total_test_data, 1)
tendency_no_hate_test = test_data[['tendency_negative']].values  # Shape (total_test_data, 1)
tendency_offensive_test = test_data[['tendency_offensive']].values  # Shape (total_test_data, 1)

# Concatenate features for the test data
X1_test = np.concatenate((X_test, tendency_hate_test, tendency_no_hate_test, tendency_offensive_test), axis=1)
print("Shape of Test Data with Tendencies:", X1_test.shape)  # Should be (total_test_data, 54) if not empty


Label Counts in Data:
class
hate_speech           15358
offensive_language     3328
neither                1140
Name: count, dtype: int64

Label Counts in Test Data:
class
hate_speech           3832
offensive_language     835
neither                290
Name: count, dtype: int64

Data DataFrame with Ratios:
                    class  ratio_positive  ratio_negative  ratio_offensive
15272             neither             0.0             0.0              0.0
9351   offensive_language             0.0             0.0              0.0
20323         hate_speech             0.0             0.0              0.0
3638          hate_speech             0.0             0.0              0.0
20579         hate_speech             0.0             0.0              0.0
...                   ...             ...             ...              ...
21575  offensive_language             0.0             0.0              0.0
5390          hate_speech             0.0             0.0              0.0
860           hat

In [ ]:
!pip install streamlit
import streamlit as st

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.5 MB/s eta 0:00:00


In [ ]:
# Assuming 'data' is your DataFrame containing 'clean_text' and 'class'
# Split the data
X_train, X_test, Y_train, Y_test = train_test_split(
    data["clean_text"],
    data["class"],
    test_size=0.2,
    random_state=42,
    stratify=data["class"]
)

# Optional: Print shapes to confirm split
print("Training set size:", len(X_train), "Testing set size:", len(X_test))
print("Classes in training set:", Y_train.value_counts())
print("Classes in testing set:", Y_test.value_counts())


Training set size: 15860 Testing set size: 3966
Classes in training set: class
hate_speech           12286
offensive_language     2662
neither                 912
Name: count, dtype: int64
Classes in testing set: class
hate_speech           3072
offensive_language     666
neither                228
Name: count, dtype: int64


In [ ]:
# Install gdown if you haven't already
!pip install gdown

# Use gdown to download the GloVe embeddings
!gdown 'https://drive.google.com/uc?id=1KXBbuSwzJoYA5ZtWspHYHZFW0Q3pHN4r'


Downloading...
From (original): https://drive.google.com/uc?id=1KXBbuSwzJoYA5ZtWspHYHZFW0Q3pHN4r
From (redirected): https://drive.google.com/uc?id=1KXBbuSwzJoYA5ZtWspHYHZFW0Q3pHN4r&confirm=t&uuid=80de2859-303f-4595-aa57-0a8e2f4e365b
To: /content/glove.840B.300d.txt
100% 5.65G/5.65G [01:23<00:00, 67.4MB/s]


In [ ]:
# If running in Google Colab
glove_file_path = '/content/glove.840B.300d.txt'

# Preview the first few lines of the GloVe file
with open(glove_file_path, 'r', encoding='utf-8') as f:
    for _ in range(5):
        print(f.readline())


, -0.082752 0.67204 -0.14987 -0.064983 0.056491 0.40228 0.0027747 -0.3311 -0.30691 2.0817 0.031819 0.013643 0.30265 0.0071297 -0.5819 -0.2774 -0.062254 1.1451 -0.24232 0.1235 -0.12243 0.33152 -0.006162 -0.30541 -0.13057 -0.054601 0.037083 -0.070552 0.5893 -0.30385 0.2898 -0.14653 -0.27052 0.37161 0.32031 -0.29125 0.0052483 -0.13212 -0.052736 0.087349 -0.26668 -0.16897 0.015162 -0.0083746 -0.14871 0.23413 -0.20719 -0.091386 0.40075 -0.17223 0.18145 0.37586 -0.28682 0.37289 -0.16185 0.18008 0.3032 -0.13216 0.18352 0.095759 0.094916 0.008289 0.11761 0.34046 0.03677 -0.29077 0.058303 -0.027814 0.082941 0.1862 -0.031494 0.27985 -0.074412 -0.13762 -0.21866 0.18138 0.040855 -0.113 0.24107 0.3657 -0.27525 -0.05684 0.34872 0.011884 0.14517 -0.71395 0.48497 0.14807 0.62287 0.20599 0.58379 -0.13438 0.40207 0.18311 0.28021 -0.42349 -0.25626 0.17715 -0.54095 0.16596 -0.036058 0.08499 -0.64989 0.075549 -0.28831 0.40626 -0.2802 0.094062 0.32406 0.28437 -0.26341 0.11553 0.071918 -0.47215 -0.18366 -0.3

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight

# Predefined variables
vocab_size = 10000  # Adjust as necessary
embedding_dim = 300  # Your GloVe embedding dimension
maxlen = 100  # Adjust maxlen as needed

# Load GloVe embeddings function
glove_file_path = '/content/glove.840B.300d.txt'
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            try:
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
            except ValueError:
                print(f"Skipping line: {line.strip()}")  # Log the skipped line
    return embeddings_index

# Load the GloVe embeddings
glove_embeddings = load_glove_embeddings(glove_file_path)

# Check the number of loaded embeddings
print(f"Loaded {len(glove_embeddings)} word vectors.")

# Prepare the embedding matrix based on the loaded GloVe embeddings
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Tokenization
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

for word, i in tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = glove_embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform labels for both training and testing labels
Y_train_enc = label_encoder.fit_transform(Y_train)
Y_test_enc = label_encoder.transform(Y_test)

# Ensure Y_train_enc and Y_test_enc are numpy arrays with integer dtype
Y_train_enc = np.array(Y_train_enc, dtype=np.int32)
Y_test_enc = np.array(Y_test_enc, dtype=np.int32)


# Calculate class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(Y_train_enc), y=Y_train_enc)
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Define the model
def create_model(vocab_size, embedding_dim, input_length):
    input_layer = Input(shape=(input_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                                weights=[embedding_matrix], trainable=False)(input_layer)
    bilstm_layer = Bidirectional(LSTM(128, return_sequences=True))(embedding_layer)
    dropout_layer = Dropout(0.5)(bilstm_layer)
    bilstm_layer2 = Bidirectional(LSTM(64))(dropout_layer)
    dropout_layer2 = Dropout(0.5)(bilstm_layer2)
    output_layer = Dense(3, activation='softmax')(dropout_layer2)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Create and compile the model
model = create_model(vocab_size, embedding_dim, maxlen)
model.compile(optimizer=Adam(learning_rate=0.0001),  # Lower learning rate
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

X_train_pad = np.array(X_train_pad, dtype=np.int32)
Y_train_enc = np.array(Y_train_enc, dtype=np.int32)


# Train the model
history = model.fit(X_train_pad, Y_train_enc, epochs=30, batch_size=32, validation_split=0.2,
                    class_weight=class_weights, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_pad, Y_test_enc)
print(f"Test loss: {test_loss}, Test accuracy: {test_accuracy}")

model.save('/content/hate_speech_detection_model.h5')



Skipping line: . . . -0.1573 -0.29517 0.30453 -0.54773 0.098293 -0.1776 0.21662 0.19261 -0.21101 0.53788 -0.047755 0.40675 0.023592 -0.32814 0.046858 0.19367 0.25565 -0.021019 -0.15957 -0.1023 0.20303 -0.043333 0.11618 -0.18486 0.0011948 -0.052301 0.34587 0.052335 0.16774 -0.21384 0.055947 0.24934 -0.12179 0.16749 0.28922 -0.033739 0.3015 -0.13241 0.092635 0.37155 -0.2884 -0.0052731 -0.001005 -0.51153 -0.28476 -0.20139 0.11837 -0.0055891 0.43604 0.16796 -0.2701 0.063957 -0.093253 -0.22079 0.36501 0.06545 0.23941 -0.19292 0.098293 0.12172 -0.1168 -0.027436 0.20507 -0.39139 -0.23111 0.46239 0.22888 -0.028415 -0.1798 0.23817 0.28093 -0.47935 0.23177 -0.35587 0.14246 0.11861 0.011018 0.091986 0.0054809 -0.39955 -0.40183 -0.10629 -0.30851 0.12383 -0.16737 -0.43569 0.4211 -0.57416 -0.19964 0.51312 0.090747 -0.21657 0.043519 0.24288 0.081134 0.49104 -0.33342 -0.31056 -0.3136 0.26931 -0.14402 0.33185 -0.21662 -0.072985 0.080603 -0.7266 -0.098385 -0.36233 -0.25346 0.1154 0.25738 0.15802 -0.1563

Test loss: 0.4326091408729553, Test accuracy: 0.848461925983429


In [ ]:
# Required imports
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pickle
import re
from nltk.corpus import stopwords
import nltk

# Download stopwords for NLTK if not already installed
nltk.download('stopwords')

# Load the saved model
model = load_model('/content/hate_speech_detection_model.h5')

# Load the tokenizer
with open('/content/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Install and import the Hugging Face `datasets` library to load data
!pip install datasets
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("tdavidson/hate_speech_offensive")

# Convert the Hugging Face Dataset to a pandas DataFrame
data = dataset['train'].to_pandas()

# Display the class distribution to understand the dataset
class_counts = data['class'].value_counts()
print("Class Counts in the Dataset:")
print(class_counts)

# Split the dataset into 80% training and 20% testing
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Preprocessing function for text cleaning
def text_processing(raw_text):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_text)  # Remove non-letters
    words = letters_only.lower().split()  # Convert to lowercase and split
    stops = set(stopwords.words("english"))  # Define stopwords
    meaningful_words = [w for w in words if w not in stops]  # Remove stopwords
    return " ".join(meaningful_words)  # Join words back into a single string

# Process text in the training and testing datasets
train_data["clean_text"] = train_data["tweet"].apply(text_processing)
test_data["clean_text"] = test_data["tweet"].apply(text_processing)

# Split the cleaned text and labels for testing
X_test = test_data["clean_text"]
Y_test = test_data["class"]

# Convert test text to sequences using tokenizer and pad them to the correct length
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=100)  # Adjust `maxlen` as per model input

# Get predictions from the model
predicted = model.predict(X_test_pad)

# Convert predictions to class labels by finding the index of the highest probability
predicted_classes = np.argmax(predicted, axis=1)

# Print the predicted class counts
print("Predicted Classes Distribution:")
print(pd.Series(predicted_classes).value_counts())

# Display the probabilities for each class for sample inspection (optional)
dataframe = pd.DataFrame(predicted, columns=['prob_class_0', 'prob_class_1', 'prob_class_2'])
dataframe['predClass'] = predicted_classes
print("\nProbabilities and Predicted Classes:")
print(dataframe.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24783 [00:00<?, ? examples/s]

Class Counts in the Dataset:
class
1    19190
2     4163
0     1430
Name: count, dtype: int64
155/155 ━━━━━━━━━━━━━━━━━━━━ 36s 228ms/step
Predicted Classes Distribution:
0    3356
2     957
1     644
Name: count, dtype: int64

Probabilities and Predicted Classes:
   prob_class_0  prob_class_1  prob_class_2  predClass
0      0.923629      0.064692      0.011679          0
1      0.879779      0.110796      0.009425          0
2      0.027485      0.029542      0.942973          2
3      0.165634      0.826099      0.008267          1
4      0.779621      0.212773      0.007606          0


In [2]:
import numpy as np
import pandas as pd
import pickle
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/hate_speech_detection_model.h5')

# Load the tokenizer
with open('/content/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Function to map model output to class labels based on user-defined threshold
def get_label_with_threshold(probabilities, threshold):
    labels = ['Hate Speech', 'Offensive Language', 'Neither']

    # Extract probabilities directly
    hate_speech_prob = probabilities[0]
    offensive_language_prob = probabilities[1]
    neither_prob = probabilities[2]

    # Determine label based on the thresholds
    if (hate_speech_prob + offensive_language_prob) > threshold:
        return "Hate Speech"
    elif neither_prob > (hate_speech_prob + offensive_language_prob) + threshold:
        return "Not Hate Speech"
    else:
        return "Uncertain Classification"

# Preprocess and vectorize function
def preprocess_and_vectorize(tweet, tokenizer, maxlen=100):
    # Vectorize the tweet
    tweet_seq = tokenizer.texts_to_sequences([tweet])
    tweet_pad = pad_sequences(tweet_seq, maxlen=maxlen)
    return tweet_pad

# Function to determine the threshold based on sensitivity option
def get_threshold(sensitivity_level):
    return (10 - sensitivity_level) / 10  # Converts to a threshold between 0.0 and 0.9

# Main loop for terminal interface
if __name__ == '__main__':
    sensitivity_options = {
        "1": ("Very Sensitive", 9.1),
        "2": ("Sensitive", 8.9),
        "3": ("Neutral", 5),
        "4": ("Less Sensitive", 1.2),
        "5": ("Nothing Hurts Me No More", 1),
    }

    while True:
        print("\nChoose your sensitivity level:")
        for key, (label, _) in sensitivity_options.items():
            print(f"{key}: {label}")

        user_choice = input("Enter the number corresponding to your choice (or 'exit' to quit): ")
        if user_choice.lower() == 'exit':
            break

        if user_choice not in sensitivity_options:
            print("Invalid choice. Please try again.")
            continue

        sensitivity_level = sensitivity_options[user_choice][1]
        threshold = get_threshold(sensitivity_level)

        print(f"Using threshold of {threshold:.2f} based on sensitivity level '{sensitivity_options[user_choice][0]}'.")

        user_input = input("Enter text (or 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break

        # Preprocess and vectorize the text using the new tokenizer
        processed_tweet = preprocess_and_vectorize(user_input, tokenizer)

        # Pass the text to the new model
        prediction_new_model = model.predict(processed_tweet)

        # Get predicted class with the threshold
        predicted_class_new_model = get_label_with_threshold(prediction_new_model[0], threshold)

        print(f"Predicted class by new model: {predicted_class_new_model}")


Choose your sensitivity level:
1: Very Sensitive
2: Sensitive
3: Neutral
4: Less Sensitive
5: Nothing Hurts Me No More
Enter the number corresponding to your choice (or 'exit' to quit): 3
Using threshold of 0.50 based on sensitivity level 'Neutral'.
Enter text (or 'exit' to quit): women should obey husbands
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
Predicted class by new model: Uncertain Classification

Choose your sensitivity level:
1: Very Sensitive
2: Sensitive
3: Neutral
4: Less Sensitive
5: Nothing Hurts Me No More


KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install Flask


In [ ]:
%%writefile app.py
from flask import Flask, render_template, request, jsonify
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import pickle
from keras.models import load_model

# Load your model and tokenizer here
model = load_model('/content/hate_speech_detection_model.h5')

# Load the tokenizer
with open('/content/tokenizer.pkl', 'rb') as f:  # Replace with your tokenizer filename
    tokenizer = pickle.load(f)

app = Flask(__name__)

# Function to map model output to class labels based on user-defined threshold
def get_label_with_threshold(probabilities, threshold):
    hate_speech_prob = probabilities[0]
    offensive_language_prob = probabilities[1]
    neither_prob = probabilities[2]

    if (hate_speech_prob + offensive_language_prob) > threshold:
        return "Hate Speech"
    elif neither_prob > (hate_speech_prob + offensive_language_prob) + threshold:
        return "Not Hate Speech"
    else:
        return "Uncertain Classification"

def preprocess_and_vectorize(text, tokenizer, maxlen=100):
    tweet_seq = tokenizer.texts_to_sequences([text])
    tweet_pad = pad_sequences(tweet_seq, maxlen=maxlen)
    return tweet_pad

def get_threshold(sensitivity_level):
    return (10 - sensitivity_level) / 10

@app.route('/')
def index():
    return '''
        <h1>Hate Speech Detection</h1>
        <form action="/predict" method="post">
            <label for="sensitivity">Choose your sensitivity level:</label>
            <input type="range" id="sensitivity" name="sensitivity" min="1" max="10" step="0.5" value="5">
            <br>
            <label for="text">Enter text:</label>
            <input type="text" id="text" name="text" placeholder="Type your text here">
            <br>
            <input type="submit" value="Submit">
        </form>
        <div id="result"></div>
    '''

@app.route('/predict', methods=['POST'])
def predict():
    text = request.form['text']
    sensitivity_level = float(request.form['sensitivity'])
    threshold = get_threshold(sensitivity_level)

    # Preprocess and vectorize the text
    processed_text = preprocess_and_vectorize(text, tokenizer)
    prediction = model.predict(processed_text)

    # Get predicted class with the threshold
    predicted_class = get_label_with_threshold(prediction[0], threshold)

    return f'Predicted class: {predicted_class}'

if __name__ == '__main__':
    app.run()


Overwriting app.py


In [ ]:
!pip install flask pyngrok keras numpy pandas


In [ ]:
!pip install pyngrok


In [ ]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token('2nL2KHLz1GLeSC9jIw3bkNExDh3_7rnirNnmMAnuAFqaBG8Az')

# Start the ngrok tunnel on port 5001
public_url = ngrok.connect(5001)
print(" * Ngrok tunnel \"{}\" -> \"http://127.0.0.1:5001\"".format(public_url))

# Run your Flask app in the background
!python app.py &


 * Ngrok tunnel "NgrokTunnel: "https://3530-34-82-3-157.ngrok-free.app" -> "http://localhost:5001"" -> "http://127.0.0.1:5001"
2024-11-11 16:45:43.720425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-11 16:45:43.843127: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-11 16:45:43.863105: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 16:45:48.961261: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 * Serving Flask app 'app'
 * Debug mode: off
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


KeyboardInterrupt: 